In [286]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

MAX_EVENT_DISPLAY = 15 # custom number of neighbors to display
TEAM_NODE_A = 1740 # home team node to focus on and see neighbors of
TEAM_NODE_B = 1455 # away team node to focus on and see neighbors of
FIG_SIZE = 10 # graph display size

In [288]:
df = pd.read_csv('data/processed/lol/events.csv')
dfLolTeams = pd.read_csv('data/processed/lol/teams_with_names.csv')
dfLolPlayers = pd.read_csv('data/processed/lol/players_with_names.csv')

df = pd.merge(df, dfLolTeams[['team_num', 'teamname']], how='left', left_on='u', right_on='team_num')
df = df.rename(columns={'teamname': 'u_name'})
df = df.drop('team_num', axis=1)
df_1 = df[df['v_type'] == 1]
df_2 = df[df['v_type'] == 2]

# Get team names in for v
df_1 = pd.merge(df_1, dfLolTeams[['team_num', 'teamname']], how='left', left_on='v', right_on='team_num')
df_1 = df_1.drop('team_num', axis=1)
df_1 = df_1.rename(columns={'teamname': 'v_name'})
# Get player names in for v
df_2 = pd.merge(df_2, dfLolPlayers[['player_num', 'playername']], how='left', left_on='v', right_on='player_num')
df_2 = df_2.drop('player_num', axis=1)
df_2 = df_2.rename(columns={'playername': 'v_name'})

df_recombined = pd.concat([df_1, df_2])
df = df_recombined.sort_values('e_idx')

,u,v,u_type,v_type,e_type,ts,e_idx,u_name,v_name
1314796,976,2428,1,2,4,1700512826,1454260,Kawaii Kiwis,Emvipi
1314797,976,2183,1,2,4,1700512826,1454261,Kawaii Kiwis,Dani
139463,976,410,1,1,4,1700512826,1454262,Kawaii Kiwis,Erfolg Esports
1314798,976,9914,1,2,4,1700512826,1454263,Kawaii Kiwis,Kurilius
1314799,410,7258,1,2,4,1700512826,1454264,Erfolg Esports,EscoX


In [306]:
focusNodes = [
    {
        'name': TEAM_NODE_A,
        'nodeList': [],
        'edgeList': []
    },
    {
        'name': TEAM_NODE_B,
        'nodeList': [],
        'edgeList': []
    }
]

nodeLabelMapper = {
    1: '  Team:  \n',
    2: '  Player:  \n'
}

edgeLabelMapper = {
    1: 'Lost',
    2: 'Won',
    3: 'Joined',
    4: 'Info'
}

# Keys the same as node type
# (1) team
# (2) player
nodeSizeMapper = {
    1: 30,
    2: 15
}

nodeColorMapper = {
    1: '#000099',
    2: '#0099FF'
}

# Keys the same as edge_type
# (1) Lost
# (2) Won
# (3) Played
# (4) Game info
edgeColorMapper = {
    1: '#FF0000',
    2: '#15B01A',
    3: '#0099FF',
    4: '#C0C0C0'
}

edgeFontSizeMapper = {
    1: 24,
    2: 24,
    3: 12,
    4: 10
}

def reformatLabel(text):
    return text.replace(' ', '\n')

def fontAdjuster(type, text):
    if type == 1:
        return 24 if len(text) < 20 else 20
    else:
        return 18 if len(text) < 20 else 14

for focusNode in focusNodes:
    selectedTeam = df.loc[df['u'] == focusNode['name'], ['u', 'v', 'u_type', 'v_type', 'e_type', 'u_name', 'v_name']].head(MAX_EVENT_DISPLAY)
    teamTypeDF = selectedTeam.drop_duplicates()
    us = list(teamTypeDF[['u', 'u_type', 'u_name']].itertuples(index=False, name=None))
    vs = list(teamTypeDF[['v', 'v_type', 'v_name']].itertuples(index=False, name=None))
    nodesList = list(list(dict.fromkeys(us + vs)))
    nodesList = list(map(lambda x: (x[0], {
        'type': x[1],
        'label': reformatLabel(x[2]),
        'title': 'TODO',
        'margin': 15,
        'color': {
            'background': nodeColorMapper[x[1]], 
            'highlight': {
                'background': nodeColorMapper[x[1]],
                'border': 'magenta'
            },
            'hover': {
                'border': 'gray'
            }
        },
        # 'physics': False,
        # 'mass': 8 if x[1] == 1 else 2,
        'mass': 5,
        'shape': 'circle',
        'font': {'size': fontAdjuster(x[1], x[2]), 'color': 'white'},
        'borderWidthSelected': 6
        }), nodesList))
    print(nodesList[:20])
    edgesWithDups = selectedTeam.groupby(selectedTeam.columns.tolist(), as_index=False).size()
    edgesWithDupsList = list(edgesWithDups[['u', 'v', 'e_type', 'size']].itertuples(index=False, name=None))
    # Change arrow display direction if edge type is 3 (to show player joining team):
    edgesWithDupsList = list(map(lambda x: (x[1] if x[2] == 3 else x[0], x[0] if x[2] == 3 else x[1], { 
        'edge_type': x[2], 
        'weight': x[3] * 3,
        'label': edgeLabelMapper[x[2]],
        'title': 'EDGE',
        'color': edgeColorMapper[x[2]],
        'font': {'size': edgeFontSizeMapper[x[2]]},
        'smooth': True
        # 'arrowSize': 5,
        # 'physics': False
        }), edgesWithDupsList))
    focusNode['nodeList'] = nodesList
    focusNode['edgeList'] = edgesWithDupsList



[(1740, {'type': 1, 'label': 'Nongshim\nRedForce\nAcademy', 'title': 'TODO', 'margin': 15, 'color': {'background': '#000099', 'highlight': {'background': '#000099', 'border': 'magenta'}, 'hover': {'border': 'gray'}}, 'mass': 5, 'shape': 'circle', 'font': {'size': 20, 'color': 'white'}, 'borderWidthSelected': 6}), (9932, {'type': 2, 'label': 'Peter', 'title': 'TODO', 'margin': 15, 'color': {'background': '#0099FF', 'highlight': {'background': '#0099FF', 'border': 'magenta'}, 'hover': {'border': 'gray'}}, 'mass': 5, 'shape': 'circle', 'font': {'size': 18, 'color': 'white'}, 'borderWidthSelected': 6}), (6746, {'type': 2, 'label': 'Bull', 'title': 'TODO', 'margin': 15, 'color': {'background': '#0099FF', 'highlight': {'background': '#0099FF', 'border': 'magenta'}, 'hover': {'border': 'gray'}}, 'mass': 5, 'shape': 'circle', 'font': {'size': 18, 'color': 'white'}, 'borderWidthSelected': 6}), (5002, {'type': 2, 'label': 'FIESTA', 'title': 'TODO', 'margin': 15, 'color': {'background': '#0099FF'

In [317]:
focusGraph = nx.MultiDiGraph()

lineWidthMapper = {
    1: 6,
    2: 6,
    3: 3,
    4: 1
}

arrowSizeMapper = {
    1: 10,
    2: 40,
    3: 12,
    4: 1
}

allNodesList = focusNodes[0]['nodeList'] + focusNodes[1]['nodeList']
allEdgesList = focusNodes[0]['edgeList'] + focusNodes[1]['edgeList']

focusGraph.add_nodes_from(allNodesList)
focusGraph.add_edges_from(allEdgesList)

options = {
    'arrowstyle': '->',
    'arrowsize': list(arrowSizeMapper[edge_type] for u, v, edge_type in list(focusGraph.edges(data='edge_type')))
}

from pyvis.network import Network
import pyvis

net = Network(
    '1500px', '1500px',
    directed=True,
    # heading='League of Legends',
    select_menu=True,
    filter_menu=True,
    # bgcolor='#222222',
    # font_color='white'
)
net.from_nx(focusGraph) # Create directly from nx graph

options = {
    'physics':{
        'barnesHut':{
            'gravitationalConstant':-10000,
            'centralGravity': 5,
            'springLength': 200,
            'springConstant': 0.7,
            'damping': 3,
            'avoidOverlap': 100
        }
    },
    'interaction':{   
        'selectConnectedEdges': True,
        'hover': True
    },
    'edges': {
        'arrowStrikethrough': False
    }
}

net.options=options

net.show('test1.html', notebook=False) # do NOT remove the notebook=False

test1.html


In [223]:
# from pyvis.network import Network

# net = Network(
#     '1500px', '1500px',
#     directed=True
# )
# net.from_nx(focusGraph) # Create directly from nx graph

# net.show('test1.html', notebook=False) # do NOT remove the notebook=False